In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import spacy
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from googlesearch import search

In [2]:
df = pd.read_csv("/Users/jacobnyamu/Desktop/Spring 2023/Advanced Programming - Python/NTerprisers /100 Corporations.csv")

In [3]:
df.pop('EntityTypeDescriptor')

0      Domestic Profit Corporation
1      Domestic Profit Corporation
2      Domestic Profit Corporation
3      Domestic Profit Corporation
4      Domestic Profit Corporation
                  ...             
95    Domestic Benefit Corporation
96    Domestic Benefit Corporation
97    Domestic Benefit Corporation
98    Domestic Benefit Corporation
99    Domestic Benefit Corporation
Name: EntityTypeDescriptor, Length: 100, dtype: object

In [4]:
df.head()

,entityname
0,"THE DOVER HOLDINGS, INC"
1,"JASON M. CHARETTE AGENCY, INC."
2,PATHWAY DRUG DEVELOPMENT CONSULTING INC.
3,"MARTY'S LOCAL, INC."
4,"WARNER FARM, INC."


In [5]:
df2 = df.truncate(before=0, after=21)

In [6]:
df2.head(26)

,entityname
0,"THE DOVER HOLDINGS, INC"
1,"JASON M. CHARETTE AGENCY, INC."
2,PATHWAY DRUG DEVELOPMENT CONSULTING INC.
3,"MARTY'S LOCAL, INC."
4,"WARNER FARM, INC."
5,"CRAFTWAY CONTRACTING, INC."
6,"TINY SPOON CHEF, INC."
7,"COMMUNITY BANCORP OF THE BERKSHIRES, INC."
8,SURGO CORPORATION
9,"HATFIELD CONSTRUCTION, INC."


In [7]:
# Iterate over each row in the data frame, perform a Google search using the row data, and print the URL of the top result    
table = []
for index, row in df2.iterrows():
    query = row['entityname'] 
    result = next(search(query, num_results=1, sleep_interval=0.5), None)
    if result:
        print(f"{index}: '{query}' found at '{result}'")
        table.append([query, result])


HTTPError: 429 Client Error: Too Many Requests for url: https://www.google.com/sorry/index?continue=https://www.google.com/search%3Fq%3DTHE%252BDOVER%252BHOLDINGS%252C%252BINC%26num%3D3%26hl%3Den%26start%3D0&hl=en&q=EhAmAEBAUPbuAHgYq3XuloO4GID9lKIGIjDDE966IbEaiRkpSyBNxdp_upG0W6oqZ0yXmic9vWVLs8oQOVLquF7ZJSmOLRShDa4yAXI

In [ ]:
urls = pd.DataFrame(table)

In [ ]:
urls.head(20)

In [ ]:
urls.set_axis(['Business', 'URL'], axis='columns', inplace=True)

In [ ]:
urls.head()

In [ ]:
# Defining keywords related to manufacturing
keywords = [
    "manufacturing", "production", "factory", "industry", "industrial", "machinery",
    "assembly", "fabrication", "plant", "workshop", "process", "machining",
    "milling", "turning", "forging", "casting", "molding", "extrusion",
    "welding", "cutting", "grinding", "lathing", "drilling", "pressing",
    "forming", "stamping", "additive", "3D printing", "injection molding",
    "blow molding", "rotational molding", "thermoforming", "vacuum forming",
    "die casting", "sand casting", "investment casting", "lost-wax casting",
    "cold rolling", "hot rolling", "slitting", "annealing", "heat treatment",
    "sheet metal", "metallurgy", "electroplating", "galvanizing", "coating",
    "powder coating", "anodizing", "painting", "finishing", "quality control",
    "CNC", "computer numerical control", "tooling", "fixtures", "jigs",
    "material handling", "automation", "robotics", "conveyor", "packaging",
    "lean manufacturing", "just-in-time", "six sigma", "continuous improvement",
    "mass production", "batch production", "job production", "assembling",
    "subassembly", "raw material", "work in progress", "finished goods",
    "inventory", "supply chain", "logistics", "procurement", "sourcing",
    "OEM", "original equipment manufacturer", "contract manufacturing",
    "outsourcing", "offshoring", "ISO 9001", "ISO 14001", "quality management",
    "environmental management", "safety", "compliance", "regulation"
]

In [ ]:
# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

# Clean the text by removing unnecessary characters
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

# Scrape the website and extract the text
def scrape_website(url):
    try:
        response = requests.get(url)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, "html.parser")
        texts = ' '.join([clean_text(p.text) for p in soup.find_all('p')])

        return texts
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return ""

# Check if the website is related to manufacturing
def is_manufacturing_website(text, keywords):
    doc = nlp(text.lower())

    for keyword in keywords:
        if keyword in doc.text:
            return True

    return False

# Creating an empty list to store the results
results = [] 
    
for website in urls.URL:
    text = scrape_website(website)
    if is_manufacturing_website(text, keywords):
        result = {"Business": urls.Business, "Manufacturing": True}
    else:
        result = {"Business": urls.Business, "Manufacturing": False}
    results.append(result)

        
urls_verified = pd.DataFrame(results)

In [ ]:
urls_verified.head(30)

In [ ]:
final_output = pd.concat([df2, urls_verified], axis=1)
final_output.head()

In [ ]:
final_output.pop('Business')
final_output.head()

In [ ]:
final_output.to_csv('Final_output_MASS.csv')